In [8]:
import random
import sys

# === Read k, t and DNA strings from file ===
def parse_input_file(path):
    with open(path, 'r') as f:
        parts = f.read().strip().split()
    k, t = map(int, parts[:2])
    dna = parts[2:]
    return k, t, dna

# === Random k‐mer selection ===
def random_kmers(k, dna):
    kmers = []
    for seq in dna:
        start = random.randint(0, len(seq) - k)
        kmers.append(seq[start:start + k])
    return kmers

# === Build profile with pseudocounts ===
def build_profile_with_pseudocounts(motifs):
    k = len(motifs[0])
    profile = {nuc: [1]*k for nuc in "ACGT"}
    for m in motifs:
        for i, nuc in enumerate(m):
            profile[nuc][i] += 1
    total = len(motifs) + 4
    for nuc in profile:
        profile[nuc] = [cnt/total for cnt in profile[nuc]]
    return profile

# === Find most probable k‐mer under a profile ===
def most_probable_kmer(text, profile, k):
    best, max_p = text[:k], -1
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        p = 1
        for j, nuc in enumerate(kmer):
            p *= profile[nuc][j]
        if p > max_p:
            best, max_p = kmer, p
    return best

# === Score motifs (lower is better) ===
def score(motifs):
    k = len(motifs[0])
    total = 0
    for i in range(k):
        cnt = {n:0 for n in "ACGT"}
        for m in motifs:
            cnt[m[i]] += 1
        total += sum(cnt.values()) - max(cnt.values())
    return total

# === One run of randomized motif search ===
def randomized_motif_search(dna, k, t):
    motifs = random_kmers(k, dna)
    best = motifs[:]
    while True:
        prof = build_profile_with_pseudocounts(motifs)
        motifs = [most_probable_kmer(seq, prof, k) for seq in dna]
        if score(motifs) < score(best):
            best = motifs[:]
        else:
            return best

# === Repeat to choose the best across many runs ===
def repeated_randomized_motif_search(dna, k, t, runs=1000):
    best = randomized_motif_search(dna, k, t)
    best_sc = score(best)
    for _ in range(runs - 1):
        m = randomized_motif_search(dna, k, t)
        s = score(m)
        if s < best_sc:
            best, best_sc = m[:], s
    return best

# === Main execution ===
if __name__ == "__main__":
    INPUT_PATH = "rosalind_ba2f.txt"  # ensure file is in your notebook folder
    k, t, dna = parse_input_file(INPUT_PATH)
    best_motifs = repeated_randomized_motif_search(dna, k, t, runs=1000)
    print("\n".join(best_motifs))


GGGAACGTTGCGTGG
AAGTGGGTGACTCGG
TGTAACGTGACTCGG
AAGAACGTGTACCGG
AAGAACGAACCTCGG
AAGAACGTGAAATGG
AAGAACGTGACTAAA
AAGAAGTGGACTCGG
AAGAACCCTACTCGG
AAGAACGTGACCTTG
ACCGACGTGACTCGG
AACCGCGTGACTCGG
AAGAACACCACTCGG
AAGTTAGTGACTCGG
GGGAACGTGACTCGA
AAGAGAATGACTCGG
AAGAGTATGACTCGG
AAGAAAACGACTCGG
AAGAACGTACGTCGG
GAGAACGTGACTCAT
